**Model Input Construction**

In [1]:
import pandas as pd

gdf = pd.read_csv("../Data/gdf.csv")
gdf.head()

,surface,round,player1,w_name,player2,l_name,height_diff,age_diff,rank_diff,rank_point_diff,...,df_diff,w_1stIn_%,l_1stIn_%,1stIn_diff,w_1stWon_%,l_1stWon_%,1stWon_diff,w_2ndWon_%,l_2ndWon_%,2ndWon_diff
0,Hard,R128,101222,Stefan Edberg,101432,Dimitri Poliakov,5.0,2.0,161.0,3687.0,...,-1.268412,75.531915,69.230769,6.301146,74.647887,59.722222,14.925665,39.130435,34.375000,4.755435
1,Hard,R128,100995,Eduardo Masso,101242,Brad Pearce,3.0,2.2,-43.0,-175.0,...,1.749590,77.956989,62.146893,15.810097,61.379310,60.909091,0.470219,65.853659,43.283582,22.570076
2,Hard,R128,101138,Pat Cash,101120,Karel Novacek,-7.0,-0.1,-52.0,-413.0,...,-2.509111,65.853659,67.816092,-1.962433,66.666667,52.542373,14.124294,64.285714,42.857143,21.428571
3,Hard,R128,100870,Christo Van Rensburg,101256,Mark Kratzmann,7.0,3.6,23.0,79.0,...,0.013028,57.894737,48.514851,9.379885,75.000000,53.061224,21.938776,68.750000,50.000000,18.750000
4,Hard,R128,101731,Nuno Marques,101416,Joao Cunha Silva,17.0,-2.4,12.0,41.0,...,0.877193,75.187970,61.111111,14.076859,73.000000,74.025974,-1.025974,48.484848,57.142857,-8.658009


### Data Doubling

In [2]:
diff_columns = gdf.columns[gdf.columns.str.contains('diff')]
diff_columns

Index(['height_diff', 'age_diff', 'rank_diff', 'rank_point_diff', 'seed_diff',
       'exp_diff', 'h2h_diff', 'past_wins_diff', 'ace_diff', 'df_diff',
       '1stIn_diff', '1stWon_diff', '2ndWon_diff'],
      dtype='object')

**w_df is from winner persepective**

In [3]:
w_df = gdf[['surface', 'round', 'player1', 'player2'] + diff_columns.to_list()].copy()
w_df['target'] = 1 #
w_df.head()

,surface,round,player1,player2,height_diff,age_diff,rank_diff,rank_point_diff,seed_diff,exp_diff,h2h_diff,past_wins_diff,ace_diff,df_diff,1stIn_diff,1stWon_diff,2ndWon_diff,target
0,Hard,R128,101222,101432,5.0,2.0,161.0,3687.0,-1.0,2.0,0,0,1.166121,-1.268412,6.301146,14.925665,4.755435,1
1,Hard,R128,100995,101242,3.0,2.2,-43.0,-175.0,0.0,2.2,0,0,1.585566,1.749590,15.810097,0.470219,22.570076,1
2,Hard,R128,101138,101120,-7.0,-0.1,-52.0,-413.0,0.0,-0.1,0,0,1.289599,-2.509111,-1.962433,14.124294,21.428571,1
3,Hard,R128,100870,101256,7.0,3.6,23.0,79.0,0.0,3.6,0,0,3.282960,0.013028,9.379885,21.938776,18.750000,1
4,Hard,R128,101731,101416,17.0,-2.4,12.0,41.0,0.0,-2.4,0,0,-0.835422,0.877193,14.076859,-1.025974,-8.658009,1


**w_df from loser perspective**

In [4]:
l_df = -1 * w_df[diff_columns]
l_df['player1'] = w_df.player2
l_df['player2'] = w_df.player1
l_df['surface'] = w_df.surface
l_df['round'] = w_df['round']
l_df['target'] = 0
l_df = l_df[w_df.columns]
l_df.head()

,surface,round,player1,player2,height_diff,age_diff,rank_diff,rank_point_diff,seed_diff,exp_diff,h2h_diff,past_wins_diff,ace_diff,df_diff,1stIn_diff,1stWon_diff,2ndWon_diff,target
0,Hard,R128,101432,101222,-5.0,-2.0,-161.0,-3687.0,1.0,-2.0,0,0,-1.166121,1.268412,-6.301146,-14.925665,-4.755435,0
1,Hard,R128,101242,100995,-3.0,-2.2,43.0,175.0,-0.0,-2.2,0,0,-1.585566,-1.749590,-15.810097,-0.470219,-22.570076,0
2,Hard,R128,101120,101138,7.0,0.1,52.0,413.0,-0.0,0.1,0,0,-1.289599,2.509111,1.962433,-14.124294,-21.428571,0
3,Hard,R128,101256,100870,-7.0,-3.6,-23.0,-79.0,-0.0,-3.6,0,0,-3.282960,-0.013028,-9.379885,-21.938776,-18.750000,0
4,Hard,R128,101416,101731,-17.0,2.4,-12.0,-41.0,-0.0,2.4,0,0,0.835422,-0.877193,-14.076859,1.025974,8.658009,0


In [5]:
df = pd.concat([w_df, l_df], ignore_index=True)
df = df.sample(df.shape[0], random_state=42, ignore_index=True)
df

,surface,round,player1,player2,height_diff,age_diff,rank_diff,rank_point_diff,seed_diff,exp_diff,h2h_diff,past_wins_diff,ace_diff,df_diff,1stIn_diff,1stWon_diff,2ndWon_diff,target
0,Clay,R32,126952,126610,-16.0,-1.7,-82.0,-3104.0,9.0,-2.0,0,-17,-24.890316,-0.609952,-11.567683,-11.739820,-11.531987,0
1,Hard,R32,103970,104022,-8.0,0.3,41.0,930.0,-11.0,2.2,0,-15,-4.139923,0.540223,11.285837,-4.195804,-8.989424,0
2,Grass,R128,101196,102394,0.0,8.6,-43.0,-184.0,0.0,4.6,0,15,4.063147,6.030021,20.678054,-0.689655,12.183236,1
3,Hard,R16,102882,102450,11.0,-2.1,-9.0,-818.0,13.0,-0.9,0,0,-2.350530,0.795564,-6.135487,-3.589986,11.929563,1
4,Grass,R64,105732,106000,13.0,1.1,6.0,31.0,0.0,2.6,0,-2,10.147462,-2.465466,-16.459754,9.782609,0.439824,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33323,Grass,R128,101731,101026,-3.0,-5.9,-29.0,-132.0,-0.0,-6.4,0,1,-3.546791,2.954498,-1.930179,-5.036232,-7.011605,0
33324,Clay,R128,102148,103096,5.0,5.0,-25.0,-343.0,32.0,-1.0,0,-8,5.333483,2.972791,-4.547830,1.874809,3.569078,1
33325,Hard,R128,104273,104122,2.0,-0.8,1.0,18.0,0.0,0.5,0,0,-2.635468,-0.812808,7.639573,2.431745,2.624385,1
33326,Hard,R64,101481,100855,0.0,-5.7,224.0,769.0,0.0,-5.7,0,6,-1.012471,5.358686,12.100259,-5.714286,5.194805,1


### Train-Test-Split

In [6]:
X = df.drop('target', axis=1)
y = df['target']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)
print(X_train.shape, X_test.shape)

(26662, 17) (6666, 17)


### One hot encoding

In [7]:
from sklearn.preprocessing import OneHotEncoder

categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns

enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

X_train_enc = enc.fit_transform(X_train[categorical_cols])
X_test_enc = enc.transform(X_test[categorical_cols])



# Converts the encoded arrays back to DataFrames
X_train_enc = pd.DataFrame(X_train_enc, columns=enc.get_feature_names_out(categorical_cols))
X_test_enc = pd.DataFrame(X_test_enc, columns=enc.get_feature_names_out(categorical_cols))

# Concatenates them with the rest of the numeric features
X_train_enc = pd.concat([X_train.drop(columns=categorical_cols).reset_index(drop=True), 
                         X_train_enc], axis=1)

X_test_enc = pd.concat([X_test.drop(columns=categorical_cols).reset_index(drop=True), 
                        X_test_enc], axis=1)

X_train_enc.shape, X_test_enc.shape

((26662, 25), (6666, 25))

In [8]:
X_test_enc

,player1,player2,height_diff,age_diff,rank_diff,rank_point_diff,seed_diff,exp_diff,h2h_diff,past_wins_diff,...,surface_Clay,surface_Grass,surface_Hard,round_F,round_QF,round_R128,round_R16,round_R32,round_R64,round_SF
0,102382,103813,3.0,7.2,-53.0,-470.0,32.0,7.7,0,-2,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,104545,104259,30.0,-1.5,10.0,420.0,9.0,1.5,-2,-17,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,103970,104607,-21.0,3.5,-24.0,-1565.0,11.0,2.8,-2,10,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,106071,104607,-0.0,-7.1,-59.0,-3923.0,7.0,-1.7,-2,-81,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,103498,103408,10.0,-0.6,-2.0,-5.0,0.0,-4.6,0,55,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6661,106432,105341,15.0,-7.5,75.0,734.0,0.0,-6.5,0,7,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6662,103694,102562,-22.0,-5.9,-57.0,-1682.0,7.0,-1.7,0,-27,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6663,105173,200005,-8.0,10.0,-7.0,-89.0,-0.0,-0.2,0,29,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6664,109739,105163,3.0,-7.0,181.0,558.0,0.0,-8.0,0,5,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [9]:
X_test_enc


,player1,player2,height_diff,age_diff,rank_diff,rank_point_diff,seed_diff,exp_diff,h2h_diff,past_wins_diff,...,surface_Clay,surface_Grass,surface_Hard,round_F,round_QF,round_R128,round_R16,round_R32,round_R64,round_SF
0,102382,103813,3.0,7.2,-53.0,-470.0,32.0,7.7,0,-2,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,104545,104259,30.0,-1.5,10.0,420.0,9.0,1.5,-2,-17,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,103970,104607,-21.0,3.5,-24.0,-1565.0,11.0,2.8,-2,10,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,106071,104607,-0.0,-7.1,-59.0,-3923.0,7.0,-1.7,-2,-81,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,103498,103408,10.0,-0.6,-2.0,-5.0,0.0,-4.6,0,55,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6661,106432,105341,15.0,-7.5,75.0,734.0,0.0,-6.5,0,7,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6662,103694,102562,-22.0,-5.9,-57.0,-1682.0,7.0,-1.7,0,-27,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6663,105173,200005,-8.0,10.0,-7.0,-89.0,-0.0,-0.2,0,29,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6664,109739,105163,3.0,-7.0,181.0,558.0,0.0,-8.0,0,5,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


### Exporting pre-processed(final data) for modeling

In [10]:
final_data = pd.concat([X_train_enc, y_train.reset_index(drop=True)], axis=1)
final_data.to_csv("../Data/final_data/train.csv", index=False)

test_data = pd.concat([X_test_enc, y_test.reset_index(drop=True)], axis=1)
test_data.to_csv("../Data/final_data/test.csv", index=False)